In [1]:
# from urllib.request import urlretrieve

# def load_data(download=True):
#     if download:
#         data_path, _ = urlretrieve(
#             "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
#             "./adult.csv"
#         )
#         print("adult dataset download")


# load_data()

downloaded


<!-- https://blog.csdn.net/qq_43745026/article/details/107219361 -->

1. 连续性数值特征，如age,最好处理
2. 二分类文本信息，二分法处理
3. 多类别文本信息

- age:连续性数值变量；可能的处理方法：分年龄段；
- workcass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov,State-gov, Without-pay, Never-worked.
 ：雇主类型，多类别，一般处理方法：化为数值类别，比如以上八个可以分别表示为1-8（仅为示例，本文并不推荐）；
- fnlwgt: 连续性数值变量；人口普查员认为观察值的人数。该变量在本文不被使用，笔者认为该特征并不重要。
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm,Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th,Preschool.受教育程度，多类别数据，处理方法同workcass；
- education-num: 连续性数值变量，受教育水平年限，一般来讲，该值越大，工资越高；
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated,Widowed, Married-spouse-absent, Married-AF-spouse.婚姻状况，多类别数据；
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial,Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical,Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv,Armed-Forces.职业，多类别数据；
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative,Unmarried.群体性关系，多类别数据；
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.种族，多类别数据，虽然美国反对种族歧视，但是实际上这个在区分美国工资的时候蛮重要；
- sex: Female, Male.性别，最简单的二分法（0&1）；
- capital-gain: 资本收益，连续数值；
- capital-loss: 资本损失，连续数值；
- hours-per-week: 工作时间，连续数值；
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada,Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China,Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico,Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti,Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia,El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.国籍，多类别数据；
- result：结果：“>50K”或“<=50K”，二分类数据，也是本文机器学习的目的（0&1）；

In [50]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

In [30]:
col_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "result"]

# assign column names
adult = pd.read_csv(
  "./adult.csv",
  names=col_names,
)
# print(adult[:10])

In [31]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  result          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [32]:
adult.apply(lambda x: np.sum(x == " ?"))

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
result               0
dtype: int64

In [33]:
adult_clean = adult.replace(regex=[r"\?|\.|\$"], value=np.nan)

In [34]:
adult_clean.apply(lambda x: np.sum(x == " ?"))

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
result            0
dtype: int64

In [35]:
# Basic statistical values ​​of each variable, such as mean, median, mode
adult_clean.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [36]:
# fill the missing value with mode

adult_clean.fillna(
    value={
        "workclass": adult_clean["workclass"].mode()[0],
        "ouccupation": adult_clean["occupation"].mode()[0],
        "native-country": adult_clean["native-country"].mode()[0],
    },
    inplace=True,
)

In [41]:
adult_clean.describe(include=["object"])

,workclass,marital-status,occupation,relationship,race,sex,native-country,result
count,32561,32561,30718,32561,32561,32561,32561,32561
unique,8,7,14,6,5,2,41,2
top,Private,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,24532,14976,4140,13193,27816,21790,29753,24720


In [38]:
# remove features
adult_clean.drop('fnlwgt',axis=1, inplace=True)         #fnlgwt
adult_clean.drop('education',axis=1, inplace=True)      #Education
adult_clean.drop('capital-gain',axis=1, inplace=True)   #Capital Gain
adult_clean.drop('capital-loss',axis=1, inplace=True)   #Capital Loss


In [39]:
pd.set_option('display.max_columns', None)
print(adult_clean.head(10))

   age          workclass  education-num          marital-status  \
0   39          State-gov             13           Never-married   
1   50   Self-emp-not-inc             13      Married-civ-spouse   
2   38            Private              9                Divorced   
3   53            Private              7      Married-civ-spouse   
4   28            Private             13      Married-civ-spouse   
5   37            Private             14      Married-civ-spouse   
6   49            Private              5   Married-spouse-absent   
7   52   Self-emp-not-inc              9      Married-civ-spouse   
8   31            Private             14           Never-married   
9   42            Private             13      Married-civ-spouse   

           occupation    relationship    race      sex  hours-per-week  \
0        Adm-clerical   Not-in-family   White     Male              40   
1     Exec-managerial         Husband   White     Male              13   
2   Handlers-cleaners   Not-i

In [43]:
workclass = adult_clean["workclass"].unique()
marital_status = adult_clean["marital-status"].unique()
occupation = adult_clean["occupation"].unique()
relationship = adult_clean["relationship"].unique()
race = adult_clean["race"].unique()
sex = adult_clean["sex"].unique()
native_country = adult_clean["native-country"].unique()
result = adult_clean["result"].unique()

In [44]:
print(workclass)
print(marital_status)
print(occupation)
print(relationship)
print(race)
print(sex)
print(native_country)
print(result)

[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' Self-emp-inc' ' Without-pay' ' Never-worked']
[' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
[' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' nan
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv']
[' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
[' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
[' Male' ' Female']
[' United-States' ' Cuba' ' Jamaica' ' India' ' Mexico' ' South'
 ' Puerto-Rico' ' Honduras' ' England' ' Canada' ' Germany' ' Iran'
 ' Philippines' ' Italy' ' Poland' ' Columbia' ' Cambodia' ' Thailand'
 ' Ecuador' ' Laos' ' Taiwan' ' Haiti' ' Portugal' ' Dominican-Republic'
 ' El-Salvador' ' France' ' Guatemala' ' China

In [45]:
# Define the mappings based on the provided categories
workclass = {
    " State-gov": 0,
    " Self-emp-not-inc": 1,
    " Private": 2,
    " Federal-gov": 3,
    " Local-gov": 4,
    " Self-emp-inc": 5,
    " Without-pay": 6,
    " Never-worked": 7,
}
maritalStatus = {
    " Never-married": 0,
    " Married-civ-spouse": 1,
    " Divorced": 2,
    " Married-spouse-absent": 3,
    " Separated": 4,
    " Married-AF-spouse": 5,
    " Widowed": 6,
}
occupation = {
    " Adm-clerical": 0,
    " Exec-managerial": 1,
    " Handlers-cleaners": 2,
    " Prof-specialty": 3,
    " Other-service": 4,
    " Sales": 5,
    " Craft-repair": 6,
    " Transport-moving": 7,
    " Farming-fishing": 8,
    " Machine-op-inspct": 9,
    " Tech-support": 10,
    " Protective-serv": 11,
    " Armed-Forces": 12,
    " Priv-house-serv": 13,
    " nan": 14,
}
relationship = {
    " Not-in-family": 0,
    " Husband": 1,
    " Wife": 2,
    " Own-child": 3,
    " Unmarried": 4,
    " Other-relative": 5,
}
race = {
    " White": 0,
    " Black": 1,
    " Asian-Pac-Islander": 2,
    " Amer-Indian-Eskimo": 3,
    " Other": 4,
}
sex = {" Male": 0, " Female": 1}
nativeCountry = {
    " United-States": 0,
    " Cuba": 1,
    " Jamaica": 2,
    " India": 3,
    " Mexico": 4,
    " South": 5,
    " Puerto-Rico": 6,
    " Honduras": 7,
    " England": 8,
    " Canada": 9,
    " Germany": 10,
    " Iran": 11,
    " Philippines": 12,
    " Italy": 13,
    " Poland": 14,
    " Columbia": 15,
    " Cambodia": 16,
    " Thailand": 17,
    " Ecuador": 18,
    " Laos": 19,
    " Taiwan": 20,
    " Haiti": 21,
    " Portugal": 22,
    " Dominican-Republic": 23,
    " El-Salvador": 24,
    " France": 25,
    " Guatemala": 26,
    " China": 27,
    " Japan": 28,
    " Yugoslavia": 29,
    " Peru": 30,
    " Outlying-US(Guam-USVI-etc)": 31,
    " Scotland": 32,
    " Trinadad&Tobago": 33,
    " Greece": 34,
    " Nicaragua": 35,
    " Vietnam": 36,
    " Hong": 37,
    " Ireland": 38,
    " Hungary": 39,
    " Holand-Netherlands": 40,
}
money = {" <=50K": 0, " >50K": 1}

# Map the columns
adult_clean["workclass"] = adult_clean["workclass"].map(workclass)
adult_clean["marital-status"] = adult_clean["marital-status"].map(maritalStatus)
adult_clean["occupation"] = adult_clean["occupation"].map(occupation)
adult_clean["relationship"] = adult_clean["relationship"].map(relationship)
adult_clean["race"] = adult_clean["race"].map(race)
adult_clean["sex"] = adult_clean["sex"].map(sex)
adult_clean["native-country"] = adult_clean["native-country"].map(nativeCountry)
    
try:
    adult_clean['result'] = adult_clean['result'].map(money)
except:
    i = 1 # exception
else:
    i = 1

In [52]:
def loadData():
    trainDataSet = pd.read_csv('./adult.csv', header=None, skiprows = 1, names=["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","result"]).head(450)

    testDataSet = pd.read_csv('./adult.csv', header=None, skiprows = 1, names=["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country"])

    return trainDataSet,testDataSet

In [ ]:
def dataCleaning(dataSet):
    #删去不相关属性
    dataSet.drop('fnlwgt',axis=1, inplace=True)         #fnlgwt
    dataSet.drop('education',axis=1, inplace=True)      #Education
    dataSet.drop('capital-gain',axis=1, inplace=True)   #Capital Gain
    dataSet.drop('capital-loss',axis=1, inplace=True)   #Capital Loss
    dataSet = dataSet.replace(' ?', np.nan)
    # 缺失值处理，采用众数替换法（mode（）方法取众数）
    dataSet.fillna(value={'workclass':dataSet['workclass'].mode()[0],   #Workclass
                          'occupation':dataSet['occupation'].mode()[0],   #Occupation  
                          'native-country':dataSet['native-country'].mode()[0]}, #Native country
              inplace = True)  
    #print(dataSet.describe())
    #print(dataSet.describe(include= ['object']))
    #离散数据连续化
    workclass = {' State-gov': 0,' Self-emp-not-inc': 1,' Private': 2,' Federal-gov': 3,' Local-gov': 4,' Self-emp-inc': 5, ' Without-pay': 6, ' Never-worked': 7}
    maritalStatus = {' Never-married': 0,' Married-civ-spouse': 1,' Divorced': 2,' Married-spouse-absent': 3, ' Separated': 4, ' Married-AF-spouse': 5, ' Widowed': 6}
    occupation = {' Adm-clerical': 0, ' Exec-managerial': 1, ' Handlers-cleaners': 2, ' Prof-specialty': 3, ' Other-service': 4, ' Sales': 5, ' Craft-repair': 6, ' Transport-moving': 7, ' Farming-fishing': 8, ' Machine-op-inspct': 9, ' Tech-support': 10, ' Protective-serv': 11,' Armed-Forces': 12, ' Priv-house-serv': 13}
    relationship = {' Not-in-family': 0, ' Husband': 1, ' Wife': 2, ' Own-child': 3, ' Unmarried': 4, ' Other-relative': 5}
    race = {' White': 0, ' Black': 1, ' Asian-Pac-Islander': 2, ' Amer-Indian-Eskimo': 3, ' Other': 4}
    sex = {' Male': 0, ' Female': 1}
    nativeCountry = {' United-States': 0, ' Cuba': 1, ' Jamaica': 2, ' India': 3, ' Mexico': 4, ' South': 5, ' Puerto-Rico': 6, ' Honduras': 7, ' England': 8, ' Canada': 9, ' Germany': 10, ' Iran': 11, ' Philippines': 12, ' Italy': 13, ' Poland': 14, ' Columbia': 15, ' Cambodia': 16, ' Thailand': 17, ' Ecuador': 18, ' Laos': 19, ' Taiwan': 20, ' Haiti': 21, ' Portugal': 22, ' Dominican-Republic': 23, ' El-Salvador': 24, ' France': 25, ' Guatemala': 26, ' China': 27, ' Japan': 28, ' Yugoslavia': 29, ' Peru': 30, ' Outlying-US(Guam-USVI-etc)': 31, ' Scotland': 32, ' Trinadad&Tobago': 33, ' Greece': 34, ' Nicaragua': 35, ' Vietnam': 36, ' Hong': 37, ' Ireland': 38, ' Hungary': 39, ' Holand-Netherlands': 40}
    money = {' <=50K': 0, ' >50K': 1}
    
    dataSet['workclass'] = dataSet['workclass'].map(workclass)
    dataSet['marital-status'] = dataSet['marital-status'].map(maritalStatus)
    dataSet['occupation'] = dataSet['occupation'].map(occupation)
    dataSet['relationship'] = dataSet['relationship'].map(relationship)
    dataSet['race'] = dataSet['race'].map(race)
    dataSet['sex'] = dataSet['sex'].map(sex)
    dataSet['native-country'] = dataSet['native-country'].map(nativeCountry)
    try:
        dataSet['money'] = dataSet['money'].map(money)
    except:
        i = 1
    else:
        i = 1
    return dataSet

In [46]:
# manhattan distance

def findDistance(oneTest, oneTrain):
    # print(oneTest.shape[0])
    distance = 0
    for i in range(oneTest.shape[0] - 1):
        distance += abs(oneTest[i] - oneTrain[i])
    return distance

In [ ]:
def knnTest(trainDataSet,testDataSet, k):
    #print(trainDataSet.loc[0])
    
    #print(testDataSet)
    testNum = testDataSet.shape[0]#测试数据行数
    trainNum = trainDataSet.shape[0]#训练数据行数
    
    predicts = np.zeros((testNum))
    #print(testDataSet.shape[1])
    for i in range(testNum):
        print('i=',i)
        #初始化
        neighbors = np.zeros(k + 2)
        distances = np.zeros(k + 2)
        for j in range(k+2):
            distances[j]=1000
        distances[0]=-1

        for j in range(trainNum):
            #print(testDataSet.loc[i],trainDataSet.loc[j])
            distance=findDistance(testDataSet.loc[i],trainDataSet.loc[j])
            #print(distance)
            index=k
            while True:
                if distance < distances[index] :
                    neighbors[index+1] = neighbors[index]
                    distances[index+1] = distances[index]
                    index-=1
                else:
                    #Insert
                    distances[index+1] = distance
                    neighbors[index+1] = j
                    break
        #投票
        labels=[]
        for j in range(k):
            index = int(neighbors[j+1])
            labels.append(int(trainDataSet.loc[index]['money']))#获得邻居的money值
            #print(int(trainDataSet.loc[index]['money']))
        counts = []
        for label in labels:
            counts.append(int(labels.count(label)))
        #print('counts=',counts)
        predicts[i]=labels[np.argmax(counts)]
        #print("The prediction[{}] = {}".format(i,predicts[i]))
    
    data = pd.DataFrame(predicts, columns=['Income'])
    data.to_csv('./201731061220.csv',index=False)
    '''   
    #基于train的精确率
    wrong=0
    for i in range(testNum):
        #print(predicts[i],testDataSet.loc[i]['money'])
        if predicts[i]!=testDataSet.loc[i]['money']:
            wrong+=1
    return (testNum-wrong)/testNum
    '''

In [53]:
trainDataSet, testDataSet = loadData()

In [66]:
salary = trainDataSet
salary2 = testDataSet
X = salary
X2 = salary2

for col in X.columns[1::]:  #循环修改数据类型的数组
    u = X[col].unique() #将当前循环的数组加上索引
    def convert(x):
        return np.argwhere(u == x)[0,0] #返回值是u中的数据等于x的索引数组，[0，0]截取索引数组的第一排第一个
    X[col] = X[col].map(convert) #索引替换映射数据

for col in X2.columns[1::]:  #循环修改数据类型的数组
    u = X2[col].unique() #将当前循环的数组加上索引
    def convert(x):
        return np.argwhere(u == x)[0,0] #返回值是u中的数据等于x的索引数组，[0，0]截取索引数组的第一排第一个
    X2[col] = X2[col].map(convert) #索引替换映射数据

X = salary.iloc[:,[0,1,3,5,6,8,9,-2,-3]] #选取所有行
X2 = salary2.iloc[:,[0,1,3,5,6,8,9,-2,-3]] #选取所有行
y = salary["result"]

for num in range(30):
    k = num + 1
    Knn = KNeighborsClassifier(n_neighbors=k)
    Knn.fit(X,y)
    print("k={},score={}".format(k,Knn.score(X,y)))

k=1,score=0.9955555555555555
k=2,score=0.8644444444444445
k=3,score=0.8888888888888888
k=4,score=0.84
k=5,score=0.8488888888888889
k=6,score=0.8044444444444444
k=7,score=0.8177777777777778
k=8,score=0.7955555555555556
k=9,score=0.8088888888888889
k=10,score=0.8022222222222222
k=11,score=0.8111111111111111
k=12,score=0.7933333333333333
k=13,score=0.8022222222222222
k=14,score=0.7955555555555556
k=15,score=0.8066666666666666
k=16,score=0.7866666666666666
k=17,score=0.7955555555555556
k=18,score=0.7777777777777778
k=19,score=0.7822222222222223
k=20,score=0.7844444444444445
k=21,score=0.7844444444444445
k=22,score=0.7844444444444445
k=23,score=0.7844444444444445
k=24,score=0.78
k=25,score=0.7777777777777778
k=26,score=0.78
k=27,score=0.7777777777777778
k=28,score=0.7777777777777778
k=29,score=0.78
k=30,score=0.7733333333333333


In [63]:
# start:end:step

print(X2.columns)
print(X2.columns[1::])
print(X2.columns[:1])

Index(['age', 'workclass', 'education', 'marital-status', 'occupation', 'race',
       'sex', 'hours-per-week', 'capital-loss'],
      dtype='object')
Index(['workclass', 'marital-status', 'race', 'hours-per-week'], dtype='object')
Index(['age'], dtype='object')
